In [64]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split

In [65]:
# 1. Generate dataset
np.random.seed(4)
X = np.random.uniform(-5, 5, 200)
epsilon = np.random.normal(0, 0.5, 200)
y = 2 * X**2 + X + epsilon

# 2. Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).view(-1, 1)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).view(-1, 1)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)


In [66]:
# 3. Function to create a polynomial regression model using PyTorch
class PolynomialRegressionModel(nn.Module):
    def __init__(self, degree):
        super(PolynomialRegressionModel, self).__init__()
        self.degree = degree
        # Creating a sequential model with a polynomial degree input layer
        self.poly = nn.Linear(degree + 1, 1)  # +1 because degree includes X^0

    def forward(self, x):
        # Compute polynomial features
        poly_features = torch.cat([x**i for i in range(self.degree + 1)], dim=1)
        return self.poly(poly_features)

In [67]:
# 4. Function to train and evaluate the model
def train_model(model, X_train, y_train, X_val, y_val, num_epochs=10000, learning_rate=0.001):
    criterion = nn.MSELoss()  # Mean Squared Error Loss
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    train_losses = []
    val_losses = []

    for _ in range(num_epochs):
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass for training set
        y_train_pred = model(X_train)
        train_loss = criterion(y_train_pred, y_train)
        train_losses.append(train_loss.item())

        # Backward pass
        train_loss.backward()
        optimizer.step()

        # Evaluate on validation set
        with torch.no_grad():
            y_val_pred = model(X_val)
            val_loss = criterion(y_val_pred, y_val)
            val_losses.append(val_loss.item())
    
    return train_losses, val_losses

In [68]:
# 5. Train models for degrees 5 and 15 and collect errors
degrees = [5, 15]
train_errors = []
val_errors = []

for degree in degrees:
    model = PolynomialRegressionModel(degree)
    train_loss, val_loss = train_model(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor)
    train_errors.append(train_loss)
    val_errors.append(val_loss)

In [69]:
# 6. Plot the errors for both models using Plotly
fig = go.Figure()

# Add traces for each degree
for i, degree in enumerate(degrees):
    fig.add_trace(go.Scatter(x=np.arange(1000), y=train_errors[i], mode='lines', name=f"Training Error (Degree {degree})", line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=np.arange(1000), y=val_errors[i], mode='lines', name=f"Validation Error (Degree {degree})", line=dict(color='red', dash='dash')))

# Update layout with titles and axis labels
fig.update_layout(
    title="Training and Validation Errors for Polynomial Regression",
    xaxis_title="Epochs",
    yaxis_title="Mean Squared Error (MSE)",
    template="ggplot2"
)

# Show the plot
fig.show()
